---
title: "ABC.4: Introduction to bash language for bioinformatics"
author: "Samuele Soraggi, Manuel Peral Vazquez"
image: ./2024-09-03-ABC4/bash.png
date: 2024-09-03
categories: [bash, command line]
description: "Slides and bash intro at the ABC.4"
eval: true
---

:::{.callout-warning}

tutorial under construction

:::

# Introduction

The difficulty of learning bash is often underestimated by others, who expect people approaching bioinformatics to learn it automatically. Here we try to put together the first basic concepts and commands.

:::{.callout-note title="Why the bash command line?"}

Using the bash command line becomes quickly essential if you are doing bioinformatics. 

First of all, you might need it to **access a computing cluster** (for example, GenomeDK at Aarhus University), since most clusters runs on a [UNIX-based operating system](https://www.hpc.iastate.edu/guides/unix-introduction), such as Linux, using a bash command line.

Just as important is the fact that on a command line you can very easily **do operations on multiple and very large files**, something you would not be able to do using, for example, `R` or `python`. Large sequences of operations can be automatized into **pipelines** (an advanced topic not for this tutorial).

With a command line you can **run many small programs, compose them together, and organize them in a chain of commands**. This type of program organization fits well with what a bioinformatics project consist of: many tools to be applied repetitevely on multiple large files, and organizing those programs in a specific sequence. An example could be aligning to a reference genome many raw bulk-RNA sequencing files: the alignment operation must be repeated many times, and when files are finished, they might need to be merged if they are from the same sample.

:::

## Some terminology

When using a UNIX operating system (Linux, MacOs), everything on your computer fits one of two categories: **processes and files**. Processes are running instances of a program, and a program is any executable file stored in your computer. A file is any collection of data (program, image, video, audio, ...).

Whenever we write a command on the terminal and press enter, we have a shell taking the code we wrote and sending it to the kernel. The **shell is the outer layer of the operating system**, which facilitates the communication to the kernel. The **kernel is the core of the operating system**, managing the computer physical components (hardware) and interfacing them with the processes that need to run. In general, any program (browser, game, ...) you open or action (moving files, renaming folders, ...) 
 you do on your computer ends up being a process managed by the kernel. This communication process is shown in @fig-shellkernel

![Communication scheme where the outer layer is a bash shell command, which the shell then communicate to the kernel, which in turn manages the hardware resources to make the program actually run. Note that there can be **many languages for the UNIX shell**: bash is the most popular, but others exist and are used (for example *zsh* on MacOs). *Figure credit: InnoKrea.*](./2024-09-03-ABC4/shellkernel.png){#fig-shellkernel}


## Efficiency and Speed

We can roughly identify various levels of efficiency, manual work, speed, number and size of handled files when working with a command line, the typical languages like R and python, or bash pipelines:


| Programming mode | Nr of files | File Size | operational speed | Manual work | 
| ---------------- | --------------- | --------------- | ----------- | -------- |
| R, python, ...   | from 1 to 10s   | small | slow | A lot |
| Command line     | from 1 to 100s  | 1-10s GB | fast | Low-moderate |
| Unix Pipeline    | from 1 to many 1000s | many TB | fast | Low |

You will see in this tutorial how we can use basic bash utilities and handle text files. Those files would take longer time to read in R and python and the code to modify them would be in general longer and slower.

:::{.callout-note title="Why is the Bash shell fast"}

One of the advantages of the bash shell language is that shell languages are already interpreted. This means that they travel directly from the shell to the kernel in @fig-shellkernel. Languages such as `R` and `python` need to be interpreted: every time you run a command, an interpreter translates this such that the kernel is able to understand the instructions to execute. This means that the python shell sends the command to an intermediate layer (the interpreter) which finally sends the program to run to the kernel. Interpreting `R`and `python` can be inefficient, because a single command has many implicit assumptions (how to use the computer memory, how to structure the data, ...) which need to be interpreted for the kernel. Other languages such as `C` are more complex to write, because the memory and the data are specified very thoroughly by the user, and the interpreter has less burden.

:::

# Slides

Our slides introducing the bash shell

&nbsp;

 <p align="center">
  <a href="https://abc.au.dk/documentation/slides/20240903-ABC.4.zip" style="background-color: #4266A1; color: #FFFFFF; padding: 30px 20px; text-decoration: none; border-radius: 5px;">
    Download Slides
  </a>
</p>

&nbsp;

# Tutorial

Here starts the tutorial. There is only one technical prerequisite, that is, you need a **Terminal**. The terminal is where you can write your commands - which are then sent to the computer to be executed.

- Mac and Linux computers already have a software called `Terminal` installed (they are both computers with UNIX-based operating systems)

- Windows have a different sort of terminal called Powershell (it is DOS-based and not UNIX-based). Please install `MobaXTerm`, then open it and click **Start Local Terminal**.

## Terminal and folders anatomy

When you work in the terminal, you will always see a prompt which starts with something of this type 

![](./2024-09-03-ABC4/shell.png)

which provides you

- username (e.g. samuele)
- computer name (e.g. D55749)
- current working directory where the user is working at the moment (e.g. `~`, which is the short form for the home directory)

In MobaXTerm you see only date, time and current working directory

### Home directory ~

The home directory, which can be written as `~`, is usually of the form `/home/username`, and is private to the user (no other users of that computer can access it).

When you open the terminal, you always start with current working directory as your home. Try to write and execute (pressing enter) the command 

```{.bash}
pwd
```

and you will see the full home path (which is the folder structure leading to your home directory)

### Current working directory (cwd)

Every command you execute refers to your cwd. For example, write

```{.bash}
ls
```

and you will see the list of files in your cwd. Try to create an empty file now with

```{.bash}
touch emptyFile.txt
```

and create a folder, which will be inside our cwd:

```{.bash}
mkdir myFolder
```

If you use again the `ls` command, the new file and folder will show in the cwd.

Now we want to download something from the internet, for which we have a download link. We are getting a raw sequenced dataset in `fastq` format, which is currently `gzip`-compressed. The `wget` command can be used for the download. Note that now we also add an option `-O` to provide the output file name as `./myFolder/data.fastq.gz`, where the dot `.` stands for the cwd, followed by `myFolder`, followed by the file name.

```{.bash}
wget https://github.com/hartwigmedical/testdata/raw/master/100k_reads_hiseq/TESTX/TESTX_H7YRLADXX_S1_L001_R1_001.fastq.gz -O ./myFolder/data.fastq.gz
```

:::{.callout-warning}

Not all utilities are installed in MobaXTerm. If you get an error, install `wget` with the command

```{.bash}
apt-get -y install wget
```

:::

### Paths and navigating the directory tree

We have already been using directories and paths a lot, so it is time to polish some definitions. Files are organized in a directory tree, which restricted to the tutorial we are running looks like @fig-dirtree. Here `home` is a root folder, which is one of the top-level folder of your computer, so it has `/` at the beginning of its name. `/home` contains a folder with your username, which is right now your cwd. Inside your cwd you have the empty file and a folder containing the data. All those folders and file are organized with a tree hierarchy, so that `/home` is the first level, `username the second`, and `myFolder` the third level. The path in the directory tree to `data.fastq.gz` is expressed as `/home/username/myFolder/data.fastq.gz`.

![Directory tree of the tutorial. The home folder is a root folder (top-level of the tree) so its name starts with the `/` symbol. Other folders and files are at subsequent branch levels of the root folder `/home`. Some folders specific to your computer might be missing from this scheme.](./2024-09-03-ABC4/treeStructure.png){#fig-dirtree width=400}

#### Absolute and relative path

The path `/home/username/myFolder/data.fastq.gz` is called **absolute** because independent of your cwd. Let's try another absolute path: the root folder `/usr` contains all executable files of the bash utilities we are using in this tutorial. Such files are in the folder `bin`. Execute

```{.bash}

ls /usr/bin/

```

The output is a long list of executable files and some folders (sometimes they also have different colors, depending on your terminal settings). If you scroll and look, you can find familiar names like `wget, ls`, and so on. Now, this path is independent of our cwd.

On the contrary, the path `./myFolder/data.tar.gz` depends on the cwd, and is equivalent to write `myFolder/data.tar.gz`, because `./` is always included by default. **All paths that do not start with a root folder are relative!**. So to look inside `myFolder`, we can both write

```{.bash}

ls myFolder/

```

and 

```{.bash}

ls ~/myFolder/

```

where we have used `~` which is the shortform for /home/username.

### Navigating folders

How to change your cwd? Simply use the command *change directory*. For example, you might want to work inside `myFolder`. Simply write

```{.bash}

cd myFolder

```

and verify with `pwd` the new working directory path. If we want to unzip the compressed data file, we simply use its relative path:

```{.bash}
gunzip data.fastq.gz
```
 
Use `ls` to verify that you have a file with name `data.fastq`.

## Working with (text) files

Many files you use in bioinformatics are nothing else than text files which are written in a specific matter. This specific way of arranging the text in the files gives you all the **file formats** you encounter when doing bioinformatics. 

:::{.callout-note}

Some file formats are encoded differently than with plain text, and cannot usually be seen with a text editor.

:::

### Less for reading files

The first thing you want to do, is to check at how the content of a file looks like. The program `less` is perfect for this purpose: you can scroll text with the arrows, and use the keyboard to do operations like searching for text or jumping to a line. Try

```{.bash}

less data.fastq

```

and see the format of your data, where each four lines provide a sequence. The very first sequence you see should be

```
@HISEQ_HU01:89:H7YRLADXX:1:1101:1116:2123 1:N:0:ATCACG
TCTGTGTAAATTACCCAGCCTCACGTATTCCTTTAGAGCAATGCAAAACAGACTAGACAAAAGGCTTTTAAAAGTCTA
ATCTGAGATTCCTGACCAAATGT
+
CCCFFFFFHHHHHJJJJJJJJJJJJHIJJJJJJJJJIJJJJJJJJJJJJJJJJJJJHIJGHJIJJIJJJJJHHHHHHH
FFFFFFFEDDEEEEDDDDDDDDD
```

Here the first line is informative of the sequence, the second is the sequence itself and then you have an empty line (symbol +), followed by the quality scores (encoded by letters following [this table](https://learn.gencore.bio.nyu.edu/ngs-file-formats/quality-scores/)).

Try to scroll in all directions to explore some lines in the file. If you want to exit from the text viewer, press `q`.

:::{.callout-tips title="Exercise" icon=false}

Try to search online how to search a specific work in a file with `less`. Then visualize the data and try to find if there is any sequence containing a sequence of ten `N`s (which is, ten missing nucleotides). Then, answer the question below 

**How long is the shortest sequence of missing nucleotides not found anywhere in the file data.fastq?**<br><br>

<html>
<form id="quizForm">
    <label>
      <input type="radio" name="q1" value="30"> 30
    </label> <br>
    <label>
      <input type="radio" name="q1" value="25"> 25
    </label> <br>
    <label>
      <input type="radio" name="q1" value="19"> 19
    </label> <br>
    <button type="button" onclick="submitQuiz1()">Submit</button>
</form>

  <script>
  function submitQuiz1() {
      var selectedOption = getSelectedOption1("q1");
      var correctAnswer = "19";

      // Display feedback
      if (selectedOption === correctAnswer) {
        alert("Correct! Good job with the bash shell!");
      } else if (selectedOption === "25") {
        alert("Nope! A bit less,");
      } else {
        alert("Nope! Much less!");
      }
    }

    function getSelectedOption1(questionName) {
      var radioButtons = document.getElementsByName(questionName);
      for (var i = 0; i < radioButtons.length; i++) {
        if (radioButtons[i].checked) {
          return radioButtons[i].value;
        }
      }
      return null;
    }
 </script>
 </html>

:::

:::{.callout-tip}

Usually, a bash shell command can be found by searching for the command + man, for example `less man`. [The manual of less](https://man7.org/linux/man-pages/man1/less.1.html) lists all the many (many many) functionalities that can be useful to explore text files, which is always a good activity to do for double checking if things are in order.

In general: **always check the manuals** or write on the prompt the option `--help` (such as `less --help`). Usually you will understand a lot of what you can do and how from that.

:::



### File properties

How big is a file? How many lines does it have? When has it been created? You can use the good old `ls` with some extra options to list all files with some extra info:

```{.bash}
ls -lah 
```

You should get something like this depending on your computer:

```
totalt 6,3M
drwxr-xr-x+ 1 au612681 UsersGrp    0 28 aug 11:38 .
drwx------+ 1 au612681 UsersGrp    0 28 aug 10:38 ..
-rw-r--r--+ 1 au612681 UsersGrp 6,3M 28 aug 11:38 data.fastq
```

the last one is clearly the data file, with creation date and size. Other infos about the file are group and user ownership and which rights we have on the files (we will not talk about these in this tutorial). 

:::{.callout-note}

Look also at the other two elements: `.` and `..`, which represents the current directory and the one above in the directory structure. Those link you to the directories, so one can always change to the directory above by using `cd ..`

**You can combine multiple times `..` to change directories multiple levels above. Where does `cd ../../` bring you to? Hint: use the terminal to try it out if you cannot remember your folder structure.**

<html>
<form id="quizForm">
    <label>
      <input type="radio" name="q2" value="a"> /home
    </label> <br>
    <label>
      <input type="radio" name="q2" value="b"> /home/myFolder
    </label> <br>
    <label>
      <input type="radio" name="q2" value="c"> /usr/bin
    </label> <br>
    <label>
      <input type="radio" name="q2" value="d"> /home/username/
    </label> <br>
    <button type="button" onclick="submitQuiz2()">Submit</button>
</form>

  <script>
  function submitQuiz2() {
      var selectedOption = getSelectedOption2("q2");
      var correctAnswer = "a";

      // Display feedback
      if (selectedOption === correctAnswer) {
        alert("Yay! You are really going ways with the shell!");
      } else{
        alert("Ups, no! How did you get into this folder!?!?!?!?");
      }
    }

    function getSelectedOption2(questionName) {
      var radioButtons = document.getElementsByName(questionName);
      for (var i = 0; i < radioButtons.length; i++) {
        if (radioButtons[i].checked) {
          return radioButtons[i].value;
        }
      }
      return null;
    }
 </script>
 </html>

:::

Now, how many lines are there in your file? The command `wc` can show that to you. It has many options, but we are going to use the one to count lines in a file. As always, look for the manual or examples to see how you can use it in other many ways.

```{.bash}
wc -l data.fastq
```

will tell you that the file has 100000 lines, so you have 25000 sequences (each sequence is defined by 4 lines). 


### Copy and Move

Now we learn a few useful commands because we want to work on more than one file. We will create multiple copies of our file. To make a copy of the data file and called the new one dataCopy.fastq, do

```{.bash}
cp data.fastq dataCopy.fastq
```

Then we will move `data.fastq`. You could move it in any folder, even in the current directory. If you move it into the current directory, you can use the `mv` command to simply change its name:

```{.bash}
mv data.fastq dataOriginal.fastq
```

If you now use `ls -lah`, you will see that you have two files of identical size and different creation dates. But... are those files identical? `diff` can tell you if you provide two file names. It will print out differences (or nothing if files are identical)

```{.bash}
diff dataOriginal.fastq dataCopy.fastq
```

### Writing on a file

You can write something on a file using `>`. This writes any output from a command which would appear on the screen into a file. For example the following commands prints out the first four lines of the data, but they will end up inside the file fourLines.fastq

```{.bash}
head -4 dataoriginal.fastq > fourLines.fastq
```

Try to use 

```{.bash}

cat fourLines.fastq

```

to see those four lines printed out directly on the screen from the file.

:::{.callout-warning}

Using again `>` to write on the same file will not add text to the file, but will overwrite it with the new text.

:::

Let's append the first four lines of the data to its copy without overwriting it. This can be done using `>>`:

```{.bash}

head -4 dataOriginal.fastq >> dataCopy.fastq

```

We want to run again the `diff` command to compare the two sequence files. **Do you need to rewrite it? Of course not!** Your bash saves the command history.

Keep `CTRL` (or `CMD` on Mac) pressed, and in the meanwhile press `r`, then start writing `diff`. The old command will show up and you can run it pressing `Enter`. Alternatively, press the arrow `UP` to see the history of commands backward.

If you run the diff command on the two files you will see an output. This shows quite a lot of information and not only the sequence which is different:

```
--- dataOriginal.fastq
+++ dataCopy.fastq
@@ -99998,3 +99998,7 @@

```

means that there are additional lines in the second file (+++), missing in `dataOriginal.fastq` (- - -), and their coordinates.

### Piping

A last important concept is the pipe. You can create small pipelines directly on the shell with the symbol `|`. Every time you use the pipe symbol, you get the output of a command and send it to the next command. For example, the `grep` command can find a pattern in a file in this way

```{.bash}
grep NNNNN dataOriginal.fastq
```

Now, what if we want to find that pattern in the first hundred sequences? Easy! We use `head`, and then pipe it into grep! When doing this, you do not need to provide a file to `grep`, and in that way it will read the output of the pipe.

```{.bash}
head -400 dataOriginal.fastq | grep NNNNN
```

That was cool! In this case we have a small output on screen, but the output can be really huge as well. Maybe it makes sense to count how many sequences `grep` can find. Well, can't really do it by hand! Pipe the result once more using `wc` as we did earlier!

```{.bash}
head -400 dataOriginal.fastq | grep NNNNN | wc -l
```

Wait a second! That sequence absolutely needs to be saved into a file. How do you do it!

<html>
<form id="quizForm">
    <label>
      <input type="radio" name="q3" value="a"> head -400 dataOriginal.fastq | grep NNNNN | sequence.fastq
    </label> <br>
    <label>
      <input type="radio" name="q3" value="b"> head -400 dataOriginal.fastq | grep NNNNN < sequence.fastq
    </label> <br>
    <label>
      <input type="radio" name="q3" value="c"> sequence.fastq < head -400 dataOriginal.fastq | grep NNNNN  
    </label> <br>
    <label>
      <input type="radio" name="q3" value="d"> head -400 dataOriginal.fastq | grep NNNNN > sequence.fastq
    </label> <br>
    <button type="button" onclick="submitQuiz3()">Submit</button>
</form>

  <script>
  function submitQuiz3() {
      var selectedOption = getSelectedOption3("q3");
      var correctAnswer = "d";

      // Display feedback
      if (selectedOption === correctAnswer) {
        alert("Great! You are going places!");
      } else{
        alert("Sorry, try again, you are not writing into a file.");
      }
    }

    function getSelectedOption3(questionName) {
      var radioButtons = document.getElementsByName(questionName);
      for (var i = 0; i < radioButtons.length; i++) {
        if (radioButtons[i].checked) {
          return radioButtons[i].value;
        }
      }
      return null;
    }
 </script>
 </html>
